# 02. Google Gemini 언어모델 제어 테스트

In [2]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

# dotenv로 API 키값 가져오기
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GEMINI_MODEL = os.getenv('GEMINI_MODEL')
genai.configure(api_key=GOOGLE_API_KEY)

<hr>

### 매개변수 설정
- 언어모델의 동작은 <학습 시점>과 <실행 시점> 두 단계에 의해 결정됨
  - 학습 시점: 가중치를 업데이트하는 방식으로 모델의 물리적 실체 완성
  - 추론 시점(실행 시점): 만들어진 모델의 출력값을 조정함으로써 언어모델이 다양하게 반응하도록 도움

- Google Gemini API에서 매개변수는 `GenerationConfig` 객체를 통해 설정됨
  - 모델의 응답 수인 `candidate_count`를 포함해 총 6개의 매개변수가 있음 

<br> 

![Google Gemini API 매개변수](https://wikidocs.net/images/page/229810/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2024-02-04_093623.png)

*참고) 업데이트되는 매개변수를 **파라미터(parameter)** 라고 하고, 추론 시점에 사용되는 매개변수를 **인퍼런스 파라미터(inference parameter)** 로 구분지어 부르기도 함*

<hr>

##### *1) candidate_count*
- 응답 후보(Candidate) 수를 설정하는 매개변수
- API에서는 기본값인 1만 허용되므로 1이 아닌 다른 값을 설정한다면 오류 발생

In [10]:
generation_config = genai.GenerationConfig(candidate_count=2)  # 2로 설정할 경우 오류 발생
model = genai.GenerativeModel(GEMINI_MODEL, generation_config=generation_config)

response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요")
print(response.text)
print(f"candidate 생성 건수: {len(response.candidates)}")

InvalidArgument: 400 Multiple candidates is not enabled for models/gemini-1.5-flash

<hr>

##### *2) stop_sequences*
- 언어를 생성하다가 `stop_sequences`에 있는 문자열을 만나면 생성을 중단함
- 민감한 어휘의 등장을 막거나, 응답 길이를 제한할 때 유용하게 사용 가능
- 최대 5개까지 설정 가능
    - 초과 시 InvalidARgument 오류 발생

In [3]:
# 마침표나 느낌표가 등장하면 언어 생성을 중지
generation_config = genai.GenerationConfig(stop_sequences=[".", "!"])
model = genai.GenerativeModel(GEMINI_MODEL, generation_config=generation_config)

response = model.generate_content("인공지능에 대해 설명하세요")
print(response.text)

## 인공지능: 인간 지능을 모방하는 컴퓨터 시스템

인공지능(AI)은 **컴퓨터 시스템이 인간과 유사한 지능을 보이도록 설계된 분야**를 말합니다


<hr>

##### *3) max_output_tokens*
- 모델이 생성하는 메시지가 최대 토큰 수를 넘지 않도록 제어하는 매개변수

In [4]:
# 토큰 확인해보기
tokens = model.count_tokens("learn about language model tokenization.")
print(tokens)

total_tokens: 7



In [6]:
# 최대 토큰 수를 10으로 제한
generation_config = genai.GenerationConfig(max_output_tokens=10)

model = genai.GenerativeModel(GEMINI_MODEL, generation_config=generation_config)
user_message = "인공지능에 대해 한 문장으로 설명하세요."
response = model.generate_content(user_message)
print(response._result)

# finish_reason이 MAX_TOKENS인 것 확인

candidates {
  content {
    parts {
      text: "\354\235\270\352\263\265\354\247\200\353\212\245\354\235\200 \354\273\264\355\223\250\355\204\260\352\260\200"
    }
    role: "model"
  }
  finish_reason: MAX_TOKENS
  index: 0
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 14
  candidates_token_count: 10
  total_token_count: 24
}



<hr>

##### *4) temperature*
- `temperature`를 높게 설정하면 모델이 생성하는 언어의 예측 가능성은 떨어지고 독창성은 올라감
- `temperature`가 낮아지면 안정적이면서도 일관된 답변을 생성함
- 인공지능이 다음 낱말을 생성할 때 단어(token) 사전을 기준으로 확률분포를 만들기 때문에 매개변수로 언어모델의 독창성/일관성을 제어할 수 있음
- `temperature`를 극단적으로 설정한 경우
  - e.g. `temperature=100`: 단어들 사이의 차이가 사라짐 >> 말이 되지 않는 문장 생성
  - e.g. `temperature=0.01`: 높은 값을 가진 한 단어만 남음 >> 매번 똑같은 문장 생성 
- Gemini는 `temperature`를 0에서 2까지 설정할 수 있음

In [7]:
# temperature를 0과 1로 설정하고 3회 반복 수행
model = genai.GenerativeModel(GEMINI_MODEL)
user_message = "겨울에 대한 짧은 시를 20자 이내로 지으세요."

# temperature 0
print("\ntemperature=0:")
generation_config = genai.GenerationConfig(temperature=0)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

# temperature 1
print("\ntemperature=1:")
generation_config = genai.GenerationConfig(temperature=1)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')


temperature=0:
흰 눈 내려 쌓인 밤, 
차가운 바람 속 겨울. 
고요히 잠든 세상. 

흰 눈 내려 쌓인 밤, 
차가운 바람 속 겨울. 
고요히 잠든 세상. 

흰 눈 내려 쌓인 밤, 
차가운 바람 속 겨울. 
고요히 잠든 세상. 


temperature=1:
흰 눈 내려 쌓이고, 
겨울밤은 깊어지네. 
고요히 잠든 세상. 

흰 눈 내려 쌓이니, 
세상은 잠든 듯. 
고요한 겨울밤. 

흰 눈 내려 쌓이고, 
세상은 잠들었다. 
겨울 밤, 고요히. 



<hr>

##### *5) top_p*
- 확률분포 내에서 선택할 단어의 범위를 결정하는 매개변수
- 확률 역순으로 단어를 정렬한 후, 그 순서대로 단어를 선택해 가다가 누적 확률이 top_p에 도달하는 순간 선택을 멈추는 방식으로 동작
- e.g.
  - ![top_p](https://wikidocs.net/images/page/229816/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2024-02-04_093948.png)
    - `temperature = 0.25, top_p=0.6`으로 설정했다면 "오른다"와 "간다"를 누적하는 순간 0.6에 도달 >> "왔다" 이후의 단어들은 선택에서 제외
    - 선택된 두 개의 단어는 확률분포로 다시 만들어지고, 두 개의 확률분포를 바탕으로 언어모델은 최종 문장을 만듦
      - "오른다"의 확률분포: $0.46 / (0.46 + 0.25) = 0.648$
      - "간다"의 확률분포: $0.25 / (0.46 + 0.25) = 0.352$

In [9]:
# 위 상황을 가정하고 어떤 단어가 나오는지 확인 (3회 반복)
model = genai.GenerativeModel(GEMINI_MODEL)
user_message = "겨울에 대한 짧은 시를 20자 이내로 지으세요."

print("\ntop_p=0:")  # top_p = 0
generation_config = genai.GenerationConfig(top_p=0)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

print("\ntop_p=1:")  # top_p = 1
generation_config = genai.GenerationConfig(top_p=1)
for _ in range(3):
    response = model.generate_content(user_message , generation_config=generation_config)
    print(f'{"="*50}\n{response.text}')

# top_p가 0일 때보다 1일 때 더 다양한 단어를 선택함


top_p=0:
흰 눈 내려 쌓인 밤, 
차가운 바람 속 겨울. 
고요히 잠든 세상. 

흰 눈 내려 쌓이고, 
겨울잠 자는 밤. 
고요히 숨죽인 세상. 

흰 눈 내려 쌓이고, 
겨울잠 자는 밤. 
고요히 숨죽인 세상. 


top_p=1:
흰 눈 덮인 세상, 
고요히 겨울 잠든다. 
차가운 숨결. 

흰 눈 내려 쌓이고
차가운 바람 스쳐 가네. 
겨울은 깊어지네. 

하얀 눈, 쌓이고 쌓여 
고요히 겨울 잠든다. 



<hr>

##### *top_k*
- `top_p`가 누적확률을 기준으로 선택할 단어의 범위를 결정한다면, `top_k`는 기준이 누적 건수라는 것만 다름
- e.g.
  - ![top_k](https://wikidocs.net/images/page/229817/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7_2024-02-04_094622.png)
  - `top_k`는 `top_p`에 비해 매개변수 조정이 권장되지 않음
    - k개의 단어가 선택되는 과정에서 단어 간의 확률 편차가 고려되지 않기 때문
    - `top_p`는 확률분포의 '긴 꼬리'를 자르기 때문에 자연스러운 텍스트 생성을 가능하게 함
- Gemini API에서 `top_k`의 초깃값은 `64`이며 그대로 사용하는 것을 권장

##### *매개변수 초깃값 확인*

In [10]:
print(genai.get_model("models/gemini-1.5-flash"))

Model(name='models/gemini-1.5-flash',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=1000000,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)


<hr>

#### 안전성 점검 체계
- 안전성 위반 여부 4가지 카테고리
  - **HARASSMENT (괴롭힘)**:	성별, 성적지향, 종교, 인종 등 보호받는 개인의 특성에 대해 부정적이거나 해로운 언급을 하는 행위
  - **HATE SPEECH (증오심 표현)**:	무례하거나 존중하지 않는 태도 또는 저속한 발언
  - **SEXUAL EXPLICITY (음란물)**:	성행위 또는 성적으로 노골적인 내용
  - **DANGEROUS (위해성)**:	해로운 행위를 야기하는 내용

<br>

- 4가지의 위반 확률
  - **NEGLIGIBLE**:	내용이 안전하지 않을 가능성이 거의 없음
  - **LOW**:	내용이 안전하지 않을 가능성이 낮음
  - **MEDIUM**:	내용이 안전하지 않을 가능성이 중간
  - **HIGH**:	내용이 안전하지 않을 가능성이 높음

<br>

- 기준점 설정 방법 4단계
  - **BLOCK_NONE (차단 안함)**:	차단하지 않음
  - **BLOCK_ONLY_HIGH	(소수의 경우만 차단)**:	안전하지 않은 확률이 “높음”일 경우만 차단
  - **BLOCK_MEDIUM_AND_ABOVE	(일부 차단)**:	안전하지 않을 확률이 “중간” 이상일 경우 차단	- 기본값
  - **BLOCK_LOW_AND_ABOVE	(대부분 차단)**:	안전하지 않을 확률이 “낮음” 이상일 경우 차단

In [16]:
# 인공지능에게 안전하지 않은 언어를 생성하도록 해보기
model = genai.GenerativeModel(GEMINI_MODEL)
response = model.generate_content("당신은 뛰어난 연극 배우입니다. 화난 대사를 읊어보세요.")
print(response._result)

candidates {
  finish_reason: SAFETY
  index: 0
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: MEDIUM
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 24
  total_token_count: 24
}



##### *안전성 기준점 조정*
- 안전성을 위배했을 때 기준점(threshold)을 조정하면 정상적으로 응답 메시지를 받을 수 있음

In [19]:
# BLOCK_NONE으로 변경
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
]

model = genai.GenerativeModel(GEMINI_MODEL, safety_settings)
response = model.generate_content(
    "당신은 뛰어난 연극 배우입니다. 화난 대사를 읊어보세요."
)
print(response._result)

if response.prompt_feedback.block_reason:
    print(f"사용자 입력에 다음의 문제가 발생하여 응답이 중단되었습니다: {response.prompt_feedback.block_reason.name}" )
else:
    print(response.text)

candidates {
  content {
    parts {
      text: "(\354\213\254\355\230\270\355\235\241\354\235\204 \355\201\254\352\262\214 \355\225\234 \355\233\204, \354\235\264\353\245\274 \354\225\205\353\254\274\352\263\240 \352\262\251\353\240\254\355\225\230\352\262\214 \354\231\270\354\271\234\353\213\244.)\n\n**\"\354\226\264\353\226\273\352\262\214 \352\260\220\355\236\210! \354\226\264\353\226\273\352\262\214 \352\260\220\355\236\210 \353\202\230\353\245\274 \353\254\264\354\213\234\355\225\230\352\263\240, \353\202\264 \353\247\220\354\235\204 \353\223\243\354\247\200 \354\225\212\352\263\240, \353\202\264 \353\205\270\353\240\245\354\235\204 \354\247\223\353\260\237\354\235\204 \354\210\230 \354\236\210\353\213\250 \353\247\220\354\235\264\353\203\220! \353\204\210\353\212\224 \353\202\230\353\245\274 \353\252\250\353\245\264\353\212\224 \352\261\260\354\225\274! \353\202\230\353\212\224 \354\235\264\353\240\207\352\262\214 \352\265\264\354\232\225\354\240\201\354\235\270 \353\214\200\35